In [4]:
# %load iris_nn_gd_tf_test.py
# Copyright (c) 2016-2017, Deogtae Kim & DTWARE Inc. All rights reserved.
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ","
# del os.environ["CUDA_VISIBLE_DEVICES"]

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import model_selection
from sklearn import metrics


tf.reset_default_graph()
tf.set_random_seed(107)

## 데이터 수집

dataset = datasets.load_iris()

In [5]:
# 데이터를 훈련 데이터와 테스트 데이터로 분할
X_data = dataset.data
y_data = dataset.target
from sklearn.preprocessing import label_binarize
y_data = np.asarray(label_binarize(y_data, [0, 1, 2]), dtype=np.float64)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_data, y_data, test_size = .3, random_state=101, stratify=y_data)  
X_train.shape, X_test.shape, y_train.shape, y_test.shape
print(np.unique(y_train, return_counts=True))
print(y_data)

(array([0., 1.]), array([210, 105], dtype=int64))


In [6]:
## 예측 모델 정의: 소프트맥스 회귀 모델 (RELU)

X = tf.placeholder(tf.float32, [None, 4])
Y = tf.placeholder(tf.float32, [None, 3])
W1 = tf.Variable(tf.random_uniform([4,100], -1.0, 1.0))     # 원래 4*3 이었는데 4*100으로 바꿈. 히든레이어 층에 100개가 됨.   
b1 = tf.Variable(tf.zeros([100]))                           # 바이어스도 100개로.
h = tf.nn.relu(tf.matmul(X, W1) + b1)                       # N*100개 벡터생성 

W2 = tf.Variable(tf.random_uniform([100,3], -1.0, 1.0))
b2 = tf.Variable(tf.zeros([3]))
# 각 데이터에 대한 각 분류별 점수
score = tf.matmul(h, W2) + b2

pred = tf.nn.softmax(score)

## 손실 함수, 정확도, 최적화 함수 정의

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(pred), reduction_indices=[1]))
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

In [ ]:
## 훈련

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
import time
start = time.time()
for epoch in range(1000):
    c, _  = sess.run([cost, train_step], feed_dict={X: X_train, Y: y_train})
    if (epoch+1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(c), 
              ', accuacy = ', '{:.9f}'.format(sess.run(accuracy, feed_dict={X: X_test, Y: y_test})))
print("훈련 시간:", time.time() - start)  

## 모델 평가
pred2 = sess.run(pred, feed_dict={X: X_train, Y: y_train})
print(pred2[0:4,])
print("혼돈 행렬:", metrics.confusion_matrix(np.argmax(y_train, axis=1), np.argmax(pred2, axis=1)), sep="\n")
#sess.close()

## 모델 평가2
pred3 = sess.run(pred, feed_dict={X: X_test, Y: y_test})
print(pred3[0:4,])
print("혼돈 행렬:", metrics.confusion_matrix(np.argmax(y_test, axis=1), np.argmax(pred3, axis=1)), sep="\n")


Epoch: 0100 cost = 0.457220972 , accuacy =  0.822222233
Epoch: 0200 cost = 0.077519700 , accuacy =  0.955555558
Epoch: 0300 cost = 0.070968963 , accuacy =  0.955555558
Epoch: 0400 cost = 0.066194929 , accuacy =  0.955555558
Epoch: 0500 cost = 0.062356114 , accuacy =  0.955555558
Epoch: 0600 cost = 0.059183273 , accuacy =  0.955555558
Epoch: 0700 cost = 0.056507364 , accuacy =  0.955555558
Epoch: 0800 cost = 0.054215264 , accuacy =  0.955555558
Epoch: 0900 cost = 0.052226260 , accuacy =  0.955555558
Epoch: 1000 cost = 0.050481256 , accuacy =  0.955555558
훈련 시간: 1.8025267124176025
[[7.5781799e-04 9.3836081e-01 6.0881361e-02]
 [9.9764472e-01 2.3552172e-03 3.3895123e-10]
 [1.0585560e-03 9.9814260e-01 7.9890055e-04]
 [1.9074009e-06 2.0735806e-02 9.7926235e-01]]
혼돈 행렬:
[[35  0  0]
 [ 0 34  1]
 [ 0  0 35]]


- 결과를 보면 1이 하나 있으므로 예측값과 실제값이 다른 경우가 딱 한번 있었다.